In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

from tqdm import tqdm
tqdm.pandas(desc="Applying function")

from shapely.geometry import Point

## Filter with the Manhattan boundary

Please do not start this part if you have the `building_manhattan.geojson` file

In [2]:
# import dataset (from https://data.cityofnewyork.us/Housing-Development/Building-Footprints/nqwf-w8eh)
gdf_building = gpd.read_file('../../data/raw/building/Building Footprints.geojson')

In [3]:
# filtering buildings with the Manhattan boundary
gdf_manhattan = gpd.read_file('../../data/raw/boundary/Borough Boundaries.geojson')
gdf_manhattan = gdf_manhattan.loc[gdf_manhattan.loc[:,'boro_name'] == 'Manhattan']

gdf_building_manhattan = gpd.sjoin(gdf_building, gdf_manhattan, how='inner', op='intersects')

c:\Users\seung\anaconda3\envs\teamgigo\Lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [4]:
gdf_building_manhattan.to_file('../../data/raw/building/building_manhattan.geojson', driver='GeoJSON')

## Calculate the nearest street

In [5]:
gdf_building_manhattan = gpd.read_file('../../data/raw/building/building_manhattan.geojson')
gdf_streets = gpd.read_file('../../data/processed/street_network.geojson')

gdf_building_manhattan = gdf_building_manhattan.drop(['shape_len','index_right', 'boro_code', 'boro_name', 'shape_area_right','shape_leng'], axis=1)

In [6]:
# unify the coordinates systems
gdf_building_manhattan = gdf_building_manhattan.to_crs(2263)
gdf_streets.crs == gdf_building_manhattan.crs

True

In [7]:
gdf_building_manhattan = gdf_building_manhattan.sjoin_nearest(gdf_streets.loc[:,['OBJECTID','StreetWidth_Min','StreetWidth_Max','POSTED_SPEED','geometry']])

# because duplicated rows created during the spatial join process, I dropped
gdf_building_manhattan = gdf_building_manhattan.drop_duplicates(subset=['base_bbl','mpluto_bbl','globalid']).drop('index_right', axis=1)

In [8]:
gdf_building_manhattan.head()

,name,base_bbl,shape_area_left,heightroof,mpluto_bbl,cnstrct_yr,globalid,lststatype,feat_code,groundelev,geomsource,bin,lstmoddate,doitt_id,geometry,OBJECTID,StreetWidth_Min,StreetWidth_Max,POSTED_SPEED
0,None,1021210037,0.0,59.722628,1021210037,1910,{A0E56BCC-A86B-4CEF-9A42-9B4ECD61743F},Constructed,2100,154,Photogramm,1062896,2017-08-22,708881,"MULTIPOLYGON (((1001211.761 244524.798, 100123...",100007,60.0,60.0,25
1,None,1008870001,0.0,155.49,1008870001,1935,{0B06F7A8-1F83-44F3-97DD-AC524A6374CA},Constructed,2100,37,Photogramm,1018457,2017-08-22,584238,"MULTIPOLYGON (((989135.970 210787.603, 989181....",101045,32.0,32.0,25
2,None,1013490035,0.0,167.21446831,1013490035,1931,{A091951D-C73E-4B77-9A61-716C666C5446},Constructed,2100,52,Photogramm,1039988,2017-08-22,66240,"MULTIPOLYGON (((994153.480 215659.003, 994165....",99804,60.0,60.0,25
3,None,1010610006,0.0,28.57,1010610006,1910,{2BE9F674-67E7-4A49-8535-F636A16D8DA5},Constructed,2100,40,Photogramm,1026714,2017-08-22,23730,"MULTIPOLYGON (((986858.328 218070.430, 986839....",103068,32.0,32.0,25
4,None,1003060013,0.0,73.13,1003060013,1900,{2403088E-77A6-4071-8B5E-D54AFBC720D9},Constructed,2100,39,Photogramm,1003956,2017-08-22,512541,"MULTIPOLYGON (((986180.396 200723.234, 986187....",92425,34.0,45.0,25


## Calculate the nearest subway station

In [9]:
df_station = pd.read_csv('../../data/raw/mta_station/MTA_Subway_Stations.csv')

In [10]:
points = [Point(float(lat),float(long)) for lat, long in zip(df_station.loc[:,'GTFS Longitude'], df_station.loc[:,'GTFS Latitude'])]
gdf_station = gpd.GeoDataFrame(df_station, geometry=points, crs=4326)
gdf_station = gdf_station.to_crs(2263)

In [11]:
gdf_building_manhattan = gdf_building_manhattan.sjoin_nearest(gdf_station.loc[:,['GTFS Stop ID','geometry']], distance_col='distance_from_station(ft)').drop('index_right', axis=1)
# because duplicated rows created during the spatial join process, I dropped
gdf_building_manhattan = gdf_building_manhattan.drop_duplicates(subset=['base_bbl','mpluto_bbl','globalid'])

In [12]:
gdf_building_manhattan = gdf_building_manhattan.drop(['name','OBJECTID','shape_area_left','lststatype','feat_code', 'groundelev','geomsource', 'lstmoddate', 'doitt_id'], axis=1)

In [13]:
gdf_building_manhattan

,base_bbl,heightroof,mpluto_bbl,cnstrct_yr,globalid,bin,geometry,StreetWidth_Min,StreetWidth_Max,POSTED_SPEED,GTFS Stop ID,distance_from_station(ft)
0,1021210037,59.722628,1021210037,1910,{A0E56BCC-A86B-4CEF-9A42-9B4ECD61743F},1062896,"MULTIPOLYGON (((1001211.761 244524.798, 100123...",60.0,60.0,25,A10,705.429228
1,1008870001,155.49,1008870001,1935,{0B06F7A8-1F83-44F3-97DD-AC524A6374CA},1018457,"MULTIPOLYGON (((989135.970 210787.603, 989181....",32.0,32.0,25,632,302.555369
2,1013490035,167.21446831,1013490035,1931,{A091951D-C73E-4B77-9A61-716C666C5446},1039988,"MULTIPOLYGON (((994153.480 215659.003, 994165....",60.0,60.0,25,F11,1378.469618
3,1010610006,28.57,1010610006,1910,{2BE9F674-67E7-4A49-8535-F636A16D8DA5},1026714,"MULTIPOLYGON (((986858.328 218070.430, 986839....",32.0,32.0,25,A25,1620.005364
4,1003060013,73.13,1003060013,1900,{2403088E-77A6-4071-8B5E-D54AFBC720D9},1003956,"MULTIPOLYGON (((986180.396 200723.234, 986187....",34.0,45.0,25,D22,292.273497
...,...,...,...,...,...,...,...,...,...,...,...,...
45397,1016000020,125.0,1016000020,2013,{EF73AB4D-5006-4406-ADF9-2FB65FC7B2C8},1089333,"MULTIPOLYGON (((998830.388 231258.315, 998700....",NaN,NaN,,226,534.072735
45398,1000920030,338.0,1000920030,2023,{5CDE3269-D9A3-4732-BBB0-AD92B46BDDF2},1091002,"MULTIPOLYGON (((982522.527 198275.305, 982453....",20.0,22.0,25,M22,311.235059
45399,1000920032,84.72,1000920032,1925,{5D66351B-DDFF-43FB-A05D-E5A3E861CE1B},1001278,"MULTIPOLYGON (((982550.394 198238.001, 982484....",22.0,24.0,25,M22,330.921614
45400,1017080003,53.48,1017080003,1900,{1B8DDD75-71EA-44FA-A2DD-213D6A98D390},1079371,"MULTIPOLYGON (((1002022.923 228852.265, 100200...",70.0,70.0,25,622,2053.559959


## Mappindg AADT 2019 ~ 2021

In [14]:
gdf_aadt_2019 = gpd.read_file('../../data/raw/aadt/aadt_2019.geojson').to_crs(2263)
gdf_aadt_2020 = gpd.read_file('../../data/raw/aadt/aadt_2020.geojson').to_crs(2263)
gdf_aadt_2021 = gpd.read_file('../../data/raw/aadt/aadt_2021.geojson').to_crs(2263)


gdf_aadt_2019 =  gdf_aadt_2019.loc[np.logical_not(gdf_aadt_2019.loc[:,'AADT'].isnull())]
gdf_aadt_2020 =  gdf_aadt_2020.loc[np.logical_not(gdf_aadt_2020.loc[:,'MAT_ALH_2020_csv_AADT'].isnull())].rename(columns={'MAT_ALH_2020_csv_AADT':'AADT'})
gdf_aadt_2021 =  gdf_aadt_2021.loc[np.logical_not(gdf_aadt_2021.loc[:,'MAT_ALH_2021_csv_AADT'].isnull())].rename(columns={'MAT_ALH_2021_csv_AADT':'AADT'})

In [15]:
def calculate_idw_aadt_average(building, gdf_aadt):
    
    distances = gdf_aadt.distance(building)
    inverse_distance_weighted_traffic = np.average(gdf_aadt['AADT'], weights = 1/(distances + 1))

    return inverse_distance_weighted_traffic

In [17]:
gdf_building_manhattan.loc[:,'idw_aadt_2019'] = gdf_building_manhattan.loc[:,'geometry'].progress_apply(lambda x: calculate_idw_aadt_average(x, gdf_aadt_2019))

Applying function: 100%|██████████| 45402/45402 [19:17<00:00, 39.22it/s]  


In [18]:
gdf_building_manhattan.loc[:,'idw_aadt_2020'] = gdf_building_manhattan.loc[:,'geometry'].progress_apply(lambda x: calculate_idw_aadt_average(x, gdf_aadt_2020))

Applying function: 100%|██████████| 45402/45402 [07:18<00:00, 103.54it/s]


In [19]:
gdf_building_manhattan.loc[:,'idw_aadt_2021'] = gdf_building_manhattan.loc[:,'geometry'].progress_apply(lambda x: calculate_idw_aadt_average(x, gdf_aadt_2021))

Applying function: 100%|██████████| 45402/45402 [07:33<00:00, 100.08it/s]


In [25]:
gdf_building_manhattan.drop('globalid', axis=1).to_crs(4326).to_file('../../data/processed/building.geojson', driver='GeoJSON')